# Quadratic programming example
**Author:** Scott Campit

This tutorial solvers an arbitrary quadratic programming (QP) problem.

In [5]:
# Import Gurobi Solver
import gurobipy as gp
from gurobipy import GRB

In [6]:
# Example taken from https://www.gurobi.com/documentation/9.0/examples/qp_py.html 
# Copyright 2020, Gurobi Optimization, LLC

# This example formulates and solves the following simple QP model:
#  minimize
#      x^2 + x*y + y^2 + y*z + z^2 + 2 x
#  subject to
#      x + 2 y + 3 z >= 4
#      x +   y       >= 1
#      x, y, z non-negative
#
# It solves it once as a continuous model, and once as an integer model.

## Solving a QP Problem
First, we need to create an object that will hold the variables and constraints. This is equivalent to a structure in MATLAB.

In [9]:
# Create model object
mdl = gp.Model("qp_example")

Using license file /home/scampit/software/gurobi901/linux64/gurobi.lic
Academic license - for non-commercial use only


Next we need to create variables, which we can do using the `addVar()` method, and we can assign the upper and lower bounds as well. 

In a large (ie genome-scale) problem, we would of course have a more elegant solution to assigning individual variables. However, because there are only 3 variables in this example, we'll assign them individually.

In [10]:
# Create variables
x = mdl.addVar(ub=1.0, lb=0.0, name='x')
y = mdl.addVar(ub=1.0, lb=0.0, name='y')
z = mdl.addVar(ub=1.0, lb=0.0, name='z')

In [11]:
# Set objective: x^2 + x*y + y^2 + y*z + z^2 + 2 x
obj = x*x + x*y + y*y + y*z + z*z + 2*x
mdl.setObjective(obj)

Let's now add our linear equations that we are using to constrain the model

In [22]:
# Add constraint: x + 2 y + 3 z <= 4
c0 = mdl.addConstr(x + 2 * y + 3 * z >= 4, "c0")

# Add constraint: x + y >= 1
c1 = mdl.addConstr(x + y >= 1, "c1")

# Add non-negativity constraints
c2 = mdl.addConstr(x >= 0, "c2")
c3 = mdl.addConstr(y >= 0, "c3")
c4 = mdl.addConstr(z >= 0, "c4")

Finally, let's solve

In [23]:
# Optimize the model
mdl.optimize()

# Get the objective value
print("Objective value: ", mdl.objVal)

# Get the RHS
print("RHS", mdl.RHS)

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 15 rows, 3 columns and 24 nonzeros
Model fingerprint: 0xc6c7063d
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 2e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 13 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 3 columns, 5 nonzeros
Presolved model has 5 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 2
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.69015022e+05 -1.71012100e+05  1.50e+03 3.33e+02  1.00e+06     0s
   1   3.60255402e+04 -3.91306233e+04  2.28e+02 3.82e+01  1.20e+05     0s
   2   4.14685168e+

# Sensitivity Analysis
First, let's get some metrics out of the model, like the reduced cost and shadow price.

In [18]:
# Get the shadow price associated with each constraint
print(mdl.Pi)

# Get the reduced cost associated with each variable
print(mdl.RC)

[0.7777777774265019, 1.9944607818577893, 0.0, 0.0, 0.0]
[0.22776144408201618, -0.8833496720106538, 4.18484965407449e-11]


Let's now modify a single variable. For instance, let's see the impact of the RHS for the first constraint on the objective coefficient if we gradually increase the value by 1.

In [47]:
# Create a new model and hide the output table
mdl2 = mdl
mdl2.setParam('OutputFlag', False)
iterations = 10

# Divide the RHS of c1 by 0.1 (relax the constraints) for 10 iterations
for i in range(0, iterations):
    c1.RHS = c1.RHS * 0.1
    mdl2.update()
    mdl2.optimize()
    print(mdl2.objVal)

2.1111111149799298
2.1111111149799298
2.1111111149799298
2.1111111149799298
2.1111111149799298
2.1111111149799298
2.1111111149799298
2.1111111149799298
2.1111111149799298
2.1111111149799298
